In [17]:
# import all required libraries

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# read the dataset
df_main = pd.read_csv("titanic.csv")
# create a copy of the main dataframe
df = df_main.copy()

### Explore dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Survived                 887 non-null    int64  
 1   Pclass                   887 non-null    int64  
 2   Name                     887 non-null    object 
 3   Sex                      887 non-null    object 
 4   Age                      887 non-null    float64
 5   Siblings/Spouses Aboard  887 non-null    int64  
 6   Parents/Children Aboard  887 non-null    int64  
 7   Fare                     887 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.6+ KB


- The column names have / and whitespace, so that can be changed
- Sex and Name are in object type, so they are probably in string

In [4]:
# Replace the column names i.e replace "/" and whitespaces to underscore for easy access of data
df.columns = df.columns.map(lambda x: x.replace('/','_').replace(' ', '_'))

In [5]:
df.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings_Spouses_Aboard',
       'Parents_Children_Aboard', 'Fare'],
      dtype='object')

In [6]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings_Spouses_Aboard,Parents_Children_Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


Name doesn't contribute anything to the model, so it can be dropped. Sex is a categorial variable so it can be converted to ordinal

In [7]:
df = df.drop("Name", axis = 1)

In [8]:
df.head()

,Survived,Pclass,Sex,Age,Siblings_Spouses_Aboard,Parents_Children_Aboard,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [10]:
# mapping male to 0 and female to 1
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

In [13]:
df['Sex'].value_counts()

0    573
1    314
Name: Sex, dtype: int64

In [14]:
df.describe()

,Survived,Pclass,Sex,Age,Siblings_Spouses_Aboard,Parents_Children_Aboard,Fare
count,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.00000
mean,0.385569,2.305524,0.354002,29.471443,0.525366,0.383315,32.30542
std,0.487004,0.836662,0.478480,14.121908,1.104669,0.807466,49.78204
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.00000
25%,0.000000,2.000000,0.000000,20.250000,0.000000,0.000000,7.92500
50%,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.45420
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.13750
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.32920


We can see that the std of the columns like Age and Fare are large than others, so we need to normalize these values. Also Survived is out target variable so we will create a separate variable for features and target

In [37]:
# check if any column have missing values
df.isna().sum()

Survived                   0
Pclass                     0
Sex                        0
Age                        0
Siblings_Spouses_Aboard    0
Parents_Children_Aboard    0
Fare                       0
dtype: int64

There are no missing values.

In [16]:
X = df.drop(['Survived'], axis = 1)
y = df['Survived']

In [20]:
# Normalization
X_standard = StandardScaler().fit_transform(X)
pd.DataFrame(X_standard).describe()

,0,1,2,3,4,5
count,8.870000e+02,8.870000e+02,8.870000e+02,8.870000e+02,8.870000e+02,8.870000e+02
mean,-2.403189e-17,2.002657e-17,2.403189e-17,1.602126e-17,-1.702259e-17,1.301727e-16
std,1.000564e+00,1.000564e+00,1.000564e+00,1.000564e+00,1.000564e+00,1.000564e+00
min,-1.561277e+00,-7.402655e-01,-2.058350e+00,-4.758557e-01,-4.749808e-01,-6.493034e-01
25%,-3.653765e-01,-7.402655e-01,-6.533568e-01,-4.758557e-01,-4.749808e-01,-4.900196e-01
50%,8.305236e-01,-7.402655e-01,-1.042546e-01,-4.758557e-01,-4.749808e-01,-3.587899e-01
75%,8.305236e-01,1.350867e+00,6.042645e-01,4.299039e-01,-4.749808e-01,-2.347391e-02
max,8.305236e-01,1.350867e+00,3.580045e+00,6.770221e+00,6.959866e+00,9.647949e+00


Now all the vales are within the range

In [21]:
# Create training and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)

### Logistic Regression

In [22]:
# Create list of hyperparameters
logistic_parameters = {'penalty': ['none', 'l2']}
regressor = LogisticRegression(random_state = 42)
regressor_grid = GridSearchCV(regressor, logistic_parameters)
regressor_grid.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegression(random_state=42),
             param_grid={'penalty': ['none', 'l2']})

In [25]:
best_params = regressor_grid.best_params_
print(regressor_grid.best_score_)
print(regressor_grid.best_estimator_.score(X_test, y_test))

0.8138148037159125
0.7471910112359551


In [36]:
# Train Logistic Regression model using the best parameters
logistic_regressor = LogisticRegression(**best_params).fit(X_train, y_train)
logistic_pred = logistic_regressor.predict(X_test)
logistic_accuracy = accuracy_score(y_test, logistic_pred)
logistic_recall = recall_score(y_test, logistic_pred)
print("The accuracy of Logistic Regression is {:.2f} and recall is {:.2f}".format(logistic_accuracy, logistic_recall))

The accuracy of Logistic Regression is 0.75 and recall is 0.55


We weill focus mostly on recall since we want to correclty predict the survivals.